## LangChain - Agent Execution Loop

In [1]:
import json
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.base import RunnableSerializable
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [4]:
@tool
def add(x: float, y: float) -> float:
    """
    Add 'x' and 'y'.
    """
    return x + y


@tool
def multiply(x: float, y: float) -> float:
    """
    Multiply 'x' and 'y'
    """
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """
    Raise 'x' to the power of 'y'
    """
    return x**y

@tool
def subtract(x: float, y: float) -> float:
    """
    Subtract 'x' from 'y'
    """
    return y - x


@tool
def final_answer(answer: str, tool_used: list[str]) -> str:
    """
    Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the 'scratchpad'
    """
    return {"answer": answer, "tools_used": tool_used}

In [5]:
tools = [final_answer, add, subtract, multiply, exponentiate]

In [6]:
name2tool = {tool.name: tool.func for tool in tools}

In [14]:
class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")
        )

    def invoke(self, input: str) -> str:
        #  invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(content=f"{tool_out}", tool_call_id=tool_call_id)
            agent_scratchpad.append(tool_exec)
            # add a print so we can see intermediate steps
            print(f"{count}: {tool_name}({tool_args})")
            count += 1
            # if the tool call is the final answer tool we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history
        final_answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])
        # return the final answer in dict form
        return json.dumps(tool_out)

In [15]:
agent_executor = CustomAgentExecutor()

In [16]:
agent_executor.invoke(input="What is 10+10?")

0: add({'x': 10, 'y': 10})
1: final_answer({'answer': '10 + 10 equals 20.', 'tool_used': ['functions.add']})


'{"answer": "10 + 10 equals 20.", "tools_used": ["functions.add"]}'

In [17]:
agent_executor.invoke(input="What is 10 * 7.4?")

0: multiply({'x': 10, 'y': 7.4})
1: final_answer({'answer': '10 multiplied by 7.4 equals 74.0.', 'tool_used': ['functions.multiply']})


'{"answer": "10 multiplied by 7.4 equals 74.0.", "tools_used": ["functions.multiply"]}'

In [18]:
agent_executor.invoke(input="What is 10 + 10 ** 2?")

0: exponentiate({'x': 10, 'y': 2})
1: add({'x': 10, 'y': 100})
2: final_answer({'answer': '10 + 10 ** 2 equals 110.', 'tool_used': ['functions.exponentiate', 'functions.add']})


'{"answer": "10 + 10 ** 2 equals 110.", "tools_used": ["functions.exponentiate", "functions.add"]}'